# 1D and 2D Driven Dampened Harmonic Oscillator


## Theory

Read about the theory of harmonic oscillators on [Wikipedia](https://en.wikipedia.org/wiki/Harmonic_oscillator) and on https://www.youtube.com/watch?v=JHF_UA0j5bg&t=294s 

### Mechanical oscillator

The case of the two dimensional mechanical oscillator leads to the following equation:

$$
m \ddot y + \mu_y \dot y + k_y y = m_y \ddot y_d
$$

$$
m \ddot x + \mu_x \dot x + k_x x = m_x \ddot x_d
$$

Where:

* $x$, $y$ is the position,
* $\dot x$ and $\ddot x$, $\dot y$ and $\ddot y$  are respectively the speed and acceleration,
* $m_x$ and $m_y$ are the mass,
* $\mu_x$, $\mu_y$ the Damping ratio
* $k_x$, $k_y$ the stiffness,
* and $\ddot x_d$, $\ddot y_d$ the driving acceleration which is null if the oscillator is free.

### Canonical equation 

Most oscilators in 1D follow the same canonical equation:

$$
\ddot x + 2 \zeta \omega_0 \dot x + \omega_0^2 x = \ddot x_d
$$

Where:

* $\omega_0$ is the undamped pulsation,
* $\zeta$ is damping ratio,
* $\ddot x_d = a_d\sin(\omega_d t)$ is the imposed acceleration.
* $F_m$ = $a_d$ here

In the case of the mechanical oscillator:

$$
\omega_0 = \sqrt{\dfrac{k}{m}}
$$

$$
\zeta = \dfrac{\mu}{2\sqrt{mk}} 
$$



In [1]:
# Imports

%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import integrate
import ipywidgets as ipw

## Coding with sinusoidal excitation force in 1D (X Axis)

$$\ddot{x}_d = F_m \sin(\omega_d t)$$

First, you need to code: the harmonic oscillator using the standarde ODE formulation:

$$\ddot{x} = - 2 \zeta \omega_0 \dot{x} - \omega_0^2 x  + F_m \sin(\omega_d t)$$

We define $$\left\{ 
             \begin{array}{ll}
             x_1 = x \\
             x_2 = \dot{x}
             \end{array}
             \right.
          $$




Like above, we can write this equation as for all $t$,

$$\dot{X}(t) = \begin{pmatrix} \dot{x} \\ 
                              - 2 \zeta \omega_0 \dot{x} - \omega_0^2 x + F_m \sin(\omega_d t) \end{pmatrix} = f\left(X, t\right)$$

# Aktuell angepasst auf Basis der TU GRAZ Simulation

http://lampx.tugraz.at/~hadley/num/apps/numerical_integration/pendulum.en.php

In [38]:
def odeDrive(X, t, zeta, omega0, omegad_omega0):
    """
    Driven Harmonic Oscillator ODE
    """
    x, dotx = X  #Input
    omegad = omegad_omega0 * omega0 #w_d besteht aus W_0 und w_0d(inputparameter)
    #ddotx = -2*zeta*omega0*dotx - omega0**2*x + F_m * np.sin(omegad * t) #DGL Gleichung
    ddotx = - 1/zeta *dotx - np.sin(x) + F_m * np.cos(omega0 * t)
    return [dotx, ddotx] #return eine Liste von x' und x''

def update(zeta = 2, omega0 = 0.67, omegad_omega0 = 1.):
    """
    Update function and calc with odeint
    """
    X0_undamped = np.zeros(2) # Anfangswerte für ungedämpften Osz. 
    X0 =  [1., 0.] # Anfangswerte für gedämpften Osz. -> initial conditions
    
    
    # The solution is an array with shape (1000 (n_t variable), 2)
    # The first column is amplitude, and the second is velocity
    sol = integrate.odeint(odeDrive, X0_undamped, t, args = (zeta, omega0, omegad_omega0)) 
    
    
    #Plotting Stuff für Osz.
    dummy = np.zeros_like(t)
    fig = plt.figure()
    line0, = plt.plot(t, dummy, label = "position")
    line0.set_ydata(sol[:, 1]) #plots the theta
    plt.grid()
    plt.ylim(-10., 10.)
    plt.xlabel("Time, $t$")
    plt.ylabel("Amplitude, $a$")
    plt.legend()
    
    # The Poincare section
    fig, ax = plt.subplots(figsize=(6, 6))
    # We will generate a solution at 16000000(4000*4000) evenly spaced samples in the interval 
    # 0 <= t <= 4000
    t_pm = np.linspace(0, 4000 * (2*np.pi) / omega0, num= 16000000) 
    
    
    #Poincare calc (4000, because of t_pm)
    xs = integrate.odeint(odeDrive, X0, t_pm, args = (zeta, omega0, omegad_omega0) )
    x = [xs[4000*i, 0] for i in range(4000)] #poincare over position
    y = [xs[4000*i, 1] for i in range(4000)] #poincare over velocity
    
    
    #Plotting Poincare map
    ax.scatter(x, y, color='blue', s=0.1)
    plt.xlabel('x', fontsize=15)
    plt.ylabel('y', fontsize=15)
    plt.tick_params(labelsize=15)
    plt.title('The Poincare section')

    plt.show()
    
    
# Parameter Antriebskraft
F_m = 0.6


# We will generate a solution at 1000 evenly spaced samples in the interval 
# 0 <= t <= 10
Nt = 1000
t = np.linspace(0., 10., Nt)


#Interactive Jupyter
ipw.interact(update, zeta = (0., 3, 0.01), 
             omega0 = (0, 2.*np.pi*5, 2.*np.pi*0.01),
             omegad_omega0 = (0.0, 10., 0.05));

interactive(children=(FloatSlider(value=2.0, description='zeta', max=3.0, step=0.01), FloatSlider(value=0.67, …

In [42]:
## Phase portrait without excitation

# Parameter canonical DGL
#omega0 = 2. * np.pi * 1.
omega0 = 0.67
#zeta   = .1
zeta = 2
ad = 1.
omegad = 2. * np.pi * 1.2

#plotting params
Nt = 1000
tf = 10.
t = np.linspace(0., tf, Nt+1)


X0_undamped = np.zeros(2) # ungedämpft
#X0 =  [1., 0.] #gedämpft
X = integrate.odeint(odeDrive, X0_undamped, t, args = (zeta, omega0, 0))



#Plotting
plt.figure()
plt.title("Phase plane ")
plt.plot(X[:,0], X[:,1])
plt.grid()
plt.xlabel("$x$, [$m$]")
plt.ylabel("$\dot{x}$, [$m.s^{-1}$] ")
plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Adding Axis Y

Equations:

$$\ddot{y}_d = F_my \sin(\omega_d t)$$

$$\ddot{y} = - 2 \zeta_y \omega_0 \dot{y} - \omega_0^2 y  + F_my \sin(\omega_d t)$$

We define $$\left\{ 
             \begin{array}{ll}
             y_1 = y \\
             y_2 = \dot{y}
             \end{array}
           \right.
          $$

Like above, we can write this equation as for all $t$,

$$\dot{Y}(t) = \begin{pmatrix} \dot{y} \\ 
                              - 2 \zeta \omega_0 \dot{y} - \omega_0^2 y + F_my \sin(\omega_d t) \end{pmatrix} = f\left(Y, t\right)$$
                              


In [4]:
def odeDrive_Y(Y, t, zeta, omega0, omegad_omega0_y):
    """
    Driven Harmonic Oscillator ODE
    """
    y, doty = Y
    omegad_y = omegad_omega0_y * omega0
    ddoty = -2*zeta*omega0*doty - omega0**2*y + F_m_y * np.sin(omegad_y * t)
    return [doty, ddoty]

def update_Y(zeta = 0.05, omega0 = 2.*np.pi, omegad_omega0_y = 1.):
    """
    Update function and calc with odeint
    """
    Y0 = np.zeros(2) # ungedämpft
    #Y0 =  [1., 0.] #gedämpft
    sol = integrate.odeint(odeDrive_Y, Y0, t, args = (zeta, omega0, omegad_omega0_y))
    
    #Plotting
    dummy = np.zeros_like(t)
    fig = plt.figure()
    line0, = plt.plot(t, dummy, label = "position")
    line0.set_ydata(sol[:, 0])
    plt.grid()
    plt.ylim(-1., 1.)
    plt.xlabel("Time, $t$")
    plt.ylabel("Amplitude, $a$")
    plt.legend()
      # The Poincare section.
    fig, ax = plt.subplots(figsize=(6, 6))
    t_pm = np.linspace(0, 4000 * (2*np.pi) / omega0, 16000000)
    xs = integrate.odeint(odeDrive_Y, Y0, t_pm, args = (zeta, omega0, omegad_omega0_y) )

    x = [xs[4000*i, 0] for i in range(4000)]
    y = [xs[4000*i, 1] for i in range(4000)]

    ax.scatter(x, y, color='blue', s=0.1)
    plt.xlabel('x', fontsize=15)
    plt.ylabel('y', fontsize=15)
    plt.tick_params(labelsize=15)
    plt.title('The Poincare section')

    plt.show()
    
    
    
    
#Driven parameter
F_m_y = 1.

#For plotting
Nt = 1000
t = np.linspace(0., 10., Nt)

#Interactive
ipw.interact(update_Y, zeta = (0., .2, 0.01), 
             omega0 = (2.*np.pi*0.5, 2.*np.pi*5, 2.*np.pi*0.01),
             omegad_omega0_y = (0.0, 10., 0.05));

interactive(children=(FloatSlider(value=0.05, description='zeta', max=0.2, step=0.01), FloatSlider(value=6.283…

In [5]:
omegad_omega0 = 1.
Y0 = np.zeros(2) # ungedämpft
#Y0 =  [1., 0.] #gedämpft
Y = integrate.odeint(odeDrive_Y, Y0, t, args = (zeta, omega0, omegad_omega0))

plt.figure()
plt.title("Phase plane with $\zeta =$ "+str(zeta))
plt.plot(Y[:,0], Y[:,1])
plt.grid()
plt.xlabel("$y$, [$m$]")
plt.ylabel("$\dot{y}$, [$m.s^{-1}$] ")
plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

 # Literatur und Ideen
 
 http://lampx.tugraz.at/~hadley/num/apps/numerical_integration/pendulum.en.php
 http://www.databookuw.com/page-2/page-21/
 PCA FFT Wavelets SINDy
 AR,ARMA, ARIMA etc